In [78]:
#@title
from __future__ import print_function
import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.datasets import cifar10
from keras.layers import Flatten

import numpy as np
import pickle as pk
from google.colab import files

batch_size = 128
num_classes = 10
epochs = 1

# the data, split between train and test sets
(X_train,y_train), (X_test,y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 17s 0us/step


In [79]:
num_classes = len(np.unique(y_train))
print(num_classes)
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
print(class_names)

10
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [95]:
# Model definition
# First define a sequential model container
model = Sequential()
# Now add subsequent layers one by one

# Block 1
model.add(Conv2D(32,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

# Block 2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

# Block 3
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
activation_15 (Activation)   (None, 15, 15, 64)        0         
__________

In [82]:
y_train_b = keras.utils.to_categorical(y_train,num_classes)
y_test_b = keras.utils.to_categorical(y_test,num_classes)
u,ind = np.unique(y_train_b,axis=0,return_index=True)
print(u.shape)
print(u)

# data normalization
X_train_n = X_train.astype('float32')
X_test_n = X_test.astype('float32')
X_train_n /= 255
X_test_n /=255

# optimizer: rms prop
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)


# model compile
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

(10, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [83]:
# Model training
model.fit(X_train_n,y_train_b,
         batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test_n, y_test_b),
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 268s 5ms/step - loss: 1.8208 - acc: 0.3508 - val_loss: 1.5733 - val_acc: 0.4431


In [101]:
scores = model.evaluate(X_test_n, y_test_b, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

RuntimeError: ignored

In [0]:
model.save_weights("CIFAR10_weights.h5")
from google.colab import files
files.download('CIFAR10_weights.h5')

In [90]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('user uploaded file {name} with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving CIFAR_weights to CIFAR_weights (2)
user uploaded file CIFAR_weights with length 5004370 bytes


In [0]:
import h5py
weights = h5py.File('CIFAR10_weights (1).h5', 'r')

In [100]:
weights_dict = {}
for layer in model.layers:
  layer_weights = np.array(layer.get_weights())
  layer_name = layer.name
  weights_dict[layer_name] = layer_weights

pk.dump(weights_dict, open('CIFAR_weights_rolled', 'wb'))

(2,)
conv2d_13


AxisError: ignored

In [0]:
files.download('CIFAR_weights')

In [0]:
m_weights = pk.load(open("CIFAR_weights (2)", 'rb'))

In [93]:
print(m_weights.keys())

dict_keys(['conv2d_5', 'activation_1', 'conv2d_6', 'activation_2', 'max_pooling2d_3', 'conv2d_7', 'activation_3', 'conv2d_8', 'activation_4', 'max_pooling2d_4', 'flatten_3', 'dense_5', 'activation_5', 'dense_6', 'activation_6'])


In [0]:
from google.colab import files
files.download('CIFAR10_Model1_Architecture.json')

In [47]:
from keras.models import model_from_json

with open('CIFAR10_Model1_Architecture.json', 'r') as f:
  model2 = model_from_json(f.read())
  
model2.load_weights('CIFAR10_weights (2).hd5')
print(model2)

In [49]:
f = open('CIFAR10_Model1_Architecture.json')
for line in f:
  print(line)

{"class_name": "Sequential", "config": {"name": "sequential_2", "layers": [{"class_name": "Conv2D", "config": {"name": "conv2d_3", "trainable": true, "batch_input_shape": [null, 28, 28, 1], "dtype": "float32", "filters": 32, "kernel_size": [3, 3], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Conv2D", "config": {"name": "conv2d_4", "trainable": true, "filters": 64, "kernel_size": [3, 3], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "activation": "relu", "use_bias": true, "kernel_initializer": {"cla